In [ ]:
from ProtDiffusion.models.autoencoder_kl_1d import AutoencoderKL1D
from ProtDiffusion.models.dit_transformer_1d import DiTTransformer1DModel
from ProtDiffusion.models.pipeline_protein import ProtDiffusionPipeline, logits_to_token_ids
from ProtDiffusion.visualization_utils import make_logoplot

from transformers import PreTrainedTokenizerFast
from diffusers import DDPMScheduler
import torch
import torch.nn.functional as F

from datasets import load_from_disk, Dataset, DatasetDict

from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO

import os

tokenizer = PreTrainedTokenizerFast.from_pretrained("/home/kkj/ProtDiffusion/ProtDiffusion/tokenizer/tokenizer_v4.2")
noise_scheduler = DDPMScheduler(num_train_timesteps=1000, clip_sample=False)
vae = AutoencoderKL1D.from_pretrained('/home/kkj/ProtDiffusion/output/EMA_VAE_v24.12')
transformer = DiTTransformer1DModel.from_pretrained('/home/kkj/ProtDiffusion/output/EMA-temp_DiT')


In [ ]:
pipeline = ProtDiffusionPipeline(
    transformer=transformer,
    vae=vae,
    scheduler=noise_scheduler,
    tokenizer=tokenizer,
)
test_dir = os.path.join("temp")
os.makedirs(test_dir, exist_ok=True)

seqs_lens = [256]
class_labels = [0]
guidance_scale = 4.0
eval_num_inference_steps = 10

output = pipeline(seq_len=seqs_lens,
                  class_labels=class_labels,
                  guidance_scale=guidance_scale,
                  num_inference_steps=eval_num_inference_steps,
                  generator=None,
                  output_type='logits',
                  return_hidden_latents=True,
)
logits = output.seqs

In [ ]:
# Plot some random latents



In [ ]:
pipeline.animate_inference(output, test_dir + "/inference")